In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import function
import pandasql
import sqlite3
from IPython.display import clear_output



In [2]:
df = pd.read_excel('VisitorEntrancesV3.xlsx')

In [3]:
df.head()

,Date,Site_Name,Israelis_Count,region_Central,region_Judea_Samaria,region_North,region_South,is_weekend,is_camping,special_activity,...,Season_summer,Season_winter,last_year_visitors,Exceeded_pm10_in_Israel,Site_Exceeded,IsHoliday,IsVacation,IsHeritage,Temperature,isHeatwave
0,2016-03-02,Akhziv,211,0,0,1,0,0,1,0,...,0,0,NaN,0,0,False,False,False,30.1,0
1,2016-03-17,Akhziv,186,0,0,1,0,0,1,0,...,0,0,NaN,0,0,False,False,False,19.2,0
2,2016-03-21,Akhziv,59,0,0,1,0,0,1,0,...,0,0,NaN,0,0,False,False,False,20.1,0
3,2016-03-25,Akhziv,6,0,0,1,0,1,1,0,...,0,0,NaN,1,0,True,True,False,26.5,0
4,2016-04-01,Akhziv,9,0,0,1,0,1,1,0,...,0,0,NaN,0,0,False,True,False,27.7,0


In [4]:
tmp_df = df[['Date','Site_Name','Israelis_Count','isHeatwave']]
totalDays = len(tmp_df.groupby('Date'))
totalDaysExceeded = (tmp_df.groupby('Date').isHeatwave.sum()>0).sum()

print('total days : ', totalDays)
print('total days exceeded : ', totalDaysExceeded)

total days :  1406
total days exceeded :  202


In [9]:
def heatWaveTest(df,site='All',date_diff_number=6):
    dataset = df
    dataset = df.loc[(df.IsHoliday==0)&(df.IsVacation==0)]
    if(site!='All'):
        dataset = df.loc[(df.Site_Name==site)&(df.IsHoliday==0)&(df.IsVacation==0)]
    tmp = dataset.sort_values(['Site_Name','Date']).reset_index()
    l_HeatWave = []
    l_HeaWave_Not = []
    data_test = pd.DataFrame()
    for i in range(date_diff_number,len(tmp)-date_diff_number):    
        prev = tmp['Date'][i-date_diff_number]
        now = tmp['Date'][i]
        if not function.date_diff(prev,now,date_diff_number):
            continue    
        prev = tmp['Date'][i]
        now = tmp['Date'][i+date_diff_number]
        if not function.date_diff(prev,now,date_diff_number):
            continue
        prev = tmp['Site_Name'][i-date_diff_number]
        now = tmp['Site_Name'][i]
        if now!=prev:
            continue
        prev = tmp['Site_Name'][i]
        now = tmp['Site_Name'][i+date_diff_number]
        if now!=prev:
            continue
        prev = tmp['isHeatwave'][i-date_diff_number:i]
        now = tmp['isHeatwave'][i:i+date_diff_number]
        # d={'prev':prev, 'now':now}
        # print(d)
        if prev.sum()>2 and now.sum()==0:
            l_HeatWave.append(tmp['Israelis_Count'][i-date_diff_number:i].sum())
            l_HeaWave_Not.append(tmp['Israelis_Count'][i:i+date_diff_number].sum())
            data_test = data_test.append({'sum week before with heatwave':tmp['Israelis_Count'][i-date_diff_number:i].sum(),
                                          'sum week after without heatwave':tmp['Israelis_Count'][i:i+date_diff_number].sum(),
                                          'start date week before':str(tmp['Date'][i-date_diff_number])+' - '+str(tmp['Date'][i-1]),
                                          'start date week after':str(tmp['Date'][i]) + ' - '+str(tmp['Date'][i+date_diff_number]),
                                          'site name': tmp['Site_Name'][i]},ignore_index=True,)

    print('length of paired data',len(l_HeatWave),len(l_HeaWave_Not))
    clear_output(wait=True) 
    data_size = len(l_HeaWave_Not)
    stat, p = stats.wilcoxon(l_HeaWave_Not, l_HeatWave,alternative='greater')
    print('HeatWave mean',np.mean(l_HeatWave))
    print('HeatWave median',np.median(l_HeatWave))
    print('HeatWave std',np.std(l_HeatWave))
    print('None HeatWave mean',np.mean(l_HeaWave_Not))
    print('None HeatWave median',np.median(l_HeaWave_Not))
    print('None HeatWave std',np.std(l_HeaWave_Not))
    Significant = 0 
    if p > 0.05:
        print('pval - ',"%.8f" %p, '- no differnce')
    else:
        Significant = 1 
        print('pval - ',"%.8f" %p, '- differnce')
    data_size = len(l_HeaWave_Not)
    heatWaveDf = pd.DataFrame()
    heatWaveDf = heatWaveDf.append({"Site":site,"Data Size":data_size,"P_Value":p,"Significant":Significant},ignore_index=True)
    return  heatWaveDf,data_test

heatWaveDf,data_test = heatWaveTest(df,date_diff_number=6)

heatWaveDf

HeatWave mean 1407.4021739130435
HeatWave median 1041.5
HeatWave std 1457.7014333157958
None HeatWave mean 1678.2608695652175
None HeatWave median 1153.0
None HeatWave std 1842.5769458864634
pval -  0.00010734 - differnce


C:\Users\Avivsa\AppData\Local\Temp\ipykernel_20760\34205748.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  heatWaveDf = heatWaveDf.append({"Site":site,"Data Size":data_size,"P_Value":p,"Significant":Significant},ignore_index=True)


,Site,Data Size,P_Value,Significant
0,All,184,0.000107,1


In [10]:
data_test = data_test.replace('00:00:00','',regex=True)
data_test

,sum week before with heatwave,sum week after without heatwave,start date week before,start date week after,site name
0,696,256,2018-10-19 - 2018-10-24,2018-10-25 - 2018-10-31,Akhziv
1,668,266,2018-10-20 - 2018-10-25,2018-10-26 - 2018-11-01,Akhziv
2,275,263,2018-10-21 - 2018-10-26,2018-10-27 - 2018-11-02,Akhziv
3,361,185,2018-10-22 - 2018-10-27,2018-10-28 - 2018-11-03,Akhziv
4,1305,1147,2018-10-19 - 2018-10-24,2018-10-25 - 2018-10-31,Amud Stream
...,...,...,...,...,...
179,1573,1159,2018-10-22 - 2018-10-27,2018-10-28 - 2018-11-03,Yehiam
180,460,301,2018-10-19 - 2018-10-24,2018-10-25 - 2018-10-31,Yehudiya
181,434,371,2018-10-20 - 2018-10-25,2018-10-26 - 2018-11-01,Yehudiya
182,379,441,2018-10-21 - 2018-10-26,2018-10-27 - 2018-11-02,Yehudiya
